## Assignment 1: Factor Analysis

In [3]:
import pandas as pd

transhydrogenase = pd.read_csv('transhydrogenase.csv.gz')
transhydrogenase.head()

X = transhydrogenase.values

In [4]:
print(X.var(axis=0))
# Normalization is required

X = (X - X.mean(axis=0)) / X.std(axis=0)
vals, vecs = np.linalg.eigh(np.cov(X.T))
vals = vals[::-1]
vecs = vecs[:, ::-1]

plt.plot(np.arange(1, 11), vals[:10])
plt.show()

scores = X @ vecs
fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plt.scatter(*scores[:, [0,1]].T, c='white')
for (x,y), t in zip(scores[:, [0,1]], AA.index):
    plt.text(x, y, t, horizontalalignment='center', verticalalignment='center')
for (x,y), v in zip(10*vecs[:, :2], AA.columns):
    plt.arrow(0, 0, x, y, color='red')
    plt.text(1.1 * x, 1.1 * y, v, color='red')
plt.show()


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read in data
data = pd.read_csv('alpha-helix-starts-4-5.dat.gz', sep='\s+', header=None).dropna()
data.head()

# Determine distance matrix
s = data[3][::50].values
n = len(s)
D = np.zeros((n, n))
T = [ [hamming(s[i], s[j]) for j in range(i+1, n)] for i in range(n-1)]

D[np.triu_indices(n, 1)] = [ j for i in T for j in i ]
D += D.T

# Perform cMDS
m = D.mean(axis=0)
A = -0.5 * ( D - m[:, None] - m[None, :] + m.mean() )

vals, vecs = np.linalg.eigh(A)
vals = vals[::-1]
vecs = vecs[:, ::-1]

In [ ]:
plt.plot(np.arange(1, 26), vals[:25])
plt.show()

In [ ]:
pcoa = vecs[:, :5] * vals[:5]**0.5
np.savetxt('alpha-helix.dat', pcoa)

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plt.scatter(*pcoa[:, [0, 2]].T, s=25, c=color(s, 'E'))
plt.show()


_Principal coordinates 0 and 1 show the helix-loop (H/\~), helix-extended (H/E) and extended-loop (E/\~) components_

_Principal coordinates 3 and 4 show a nice helix-turn component_

_A principal coordinate is a direction in a (here fictive) variable space such that the distance structure of the observations corresponds as closely as possible to the distance matrix of Hamming distances. The components correspond to transitions from all helix to all loop, from all helix to all turn, etc, and are governed by the structure types that occur most._


## Assignment 2: Multivariate Linear Regression.

Use the coordinate data from the same data file (`alpha-helix-starts-4-5.dat`) and the principal coordinates from Assignment 1. If doing MDS didn't work out, you can load the same data from `alpha-helix-pcoa.dat`. The principal coordinates in that file correspond to the slice \[::50\] from the original data.

Assignment:

- Perform multivariate linear regression to construct a model for predicting the principal coordinates (from sequences) from the coordinates. Note that some coordinates were set to 0 during alignment. These have 0 variance, and should be excluded.

- Plot the _estimated_ principal coordinates against the _actual_ principal coordinates. 

- Reflect on the quality of the regression model and the possibility to predict the structure classification from the coordinates using regression.

Assessment:

- Correct application of MDS
- Proper graphical representations
- Assessment and interpretation of regression
- Interpretation of and reflection on results

In [ ]:
xyz = data.iloc[::50, 4:].values
xyz = xyz[:, xyz.var(axis=0) > 0]
X = np.concatenate((np.ones((xyz.shape[0], 1)), xyz), axis=1)

B = np.linalg.inv(X.T @ X) @ X.T @ pcoa

estimates = X @ B
errors = pcoa - estimates

for i in range(pcoa.shape[1]):
    plt.scatter(estimates[:, i], pcoa[:, i])
plt.show()

_The variation of the estimates is greater than the domain of the regressor! This is a terrible model. It's not possible to shortcut the assignments of structures from the coordinates using regression._

## Assignment 3: Factor Analysis

The file `AA.dat` contains physical and chemical parameters for a range of natural and unnatural amino acids. The properties are:

- MW: Molecular weight (size)
- TL1-7: ?
- vdW: Van-der-Waals volume
- logP: Oil-water partition coefficient (hydrophobicity)
- EHOMO: Highest occupied molecular orbital
- ELUMO: Lowest unoccupied molecular orbital
- HOF: ?
- POLAR: Polarity
- EN: ?
- HA: ?
- Stot: Total surface area
- Spol: Polar surface area
- Snp: Non-polar surface area
- HDONR: Is-a-hydrogen-donor
- HACCR: Is-a-hydrogen-acceptor
- Chpos: Positive charge
- Chneg: Negative charge

Assignment:

- Perform Factor Analysis on the data
- Plot the results as a biplot
- Interpret the results and give an interpretation of the factors/components

Assessment:

- Correct application of Factor Analysis with justification of choices made
- Proper graphical representations
- Assessment and interpretation of regression
- Interpretation of and reflection on results

In [ ]:
AA = pd.read_csv('AA.dat', sep='\s+')
X = AA.values

print(AA)

In [ ]:
print(X.var(axis=0))
# Normalization is required

X = (X - X.mean(axis=0)) / X.std(axis=0)
vals, vecs = np.linalg.eigh(np.cov(X.T))
vals = vals[::-1]
vecs = vecs[:, ::-1]

plt.plot(np.arange(1, 11), vals[:10])
plt.show()

scores = X @ vecs
fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plt.scatter(*scores[:, [0,1]].T, c='white')
for (x,y), t in zip(scores[:, [0,1]], AA.index):
    plt.text(x, y, t, horizontalalignment='center', verticalalignment='center')
for (x,y), v in zip(10*vecs[:, :2], AA.columns):
    plt.arrow(0, 0, x, y, color='red')
    plt.text(1.1 * x, 1.1 * y, v, color='red')
plt.show()


In [ ]:
def varimax(components, gamma=1, maxiter=20, tol=1e-8):
    """Perform VariMax (gamma=1) or OrthoMax (gamma=0) rotation on components"""
    p,k = components.shape
    R   = np.eye(k)
    f   = float(gamma)/p
    d   = 0
    for i in range(maxiter):
        d_old = d
        L = np.dot(components, R)
        A = L**3 - f * (L*(L**2).sum(axis=0))**2
        U,s,V = np.linalg.svd(np.dot(components.T,A))
        R = np.dot(U,V)
        d = sum(s)
        if (d - d_old)**2 < tol: 
            break
    return np.dot(components, R)

V = varimax(vecs[:, :4])
F = X @ V

for i, v in zip(AA.columns, V):
    print(i, v.round(2), sep='\t')

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect(1)
plt.scatter(*F[:, [0,1]].T, c='#999999')
for (x,y), t in zip(F[:20, [0,1]], AA.index):
    plt.text(x, y, t, horizontalalignment='center', verticalalignment='center', fontsize=20)
for (x,y), v in zip(10*V[:, :2], AA.columns):
    plt.arrow(0, 0, x, y, color='red')
    plt.text(1.1 * x, 1.1 * y, v, color='red')
plt.show()

The data need to be normalized prior to PCA, because the range of variances is humongous and the variables are completely heterogenous.

The scree plot shows that there is not a clear break point, so it's difficult to chose a number of components. Four components capture 21/23 * 100% = 91.3% of the correlation, so that seems a proper number.

The varimax rotation results in the first component to be associated with the TL parameters (forgot what those are), the hydrophobicity and the polarity. The second component seems to reflect the size (MW, vdW, Stot).



***

<div class="alert alert-warning">
<b>After finishing:</b>
<ol><li>
Rename your notebook with your name and student number, like `JohnDoe_123456`, using the menu option `File` > `Rename`.
</li><li>
Evaluate the notebook by means of the menu option `Kernel` > `Restart & Run All` and check that your notebook runs without errors.
</li><li>
Save the evaluated notebook using the menu option `File` > `Save and Checkpoint`.
</li><li>
Open a terminal, change to the desktop folder using `cd ~/Desktop`, and submit your work using a command like `submit_your_work 123456 JohnDoe_123456.ipynb` (enter `submit_your_work --help` to get help on submit script usage).
</li><li>
After getting clearance from the person monitoring the exam, you are free to leave.
</li></ol>
</div>

#### Success!